<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


There is a nice pdf file with a tutorial about Julia online. It is available at: https://www.sas.upenn.edu/~jesusfv/Chapter_HPC_8_Julia.pdf


# Packages
A package is a code that extends the basic capabilities Julia
with additional functions, data structures, etc. In such a way, Julia follows the modern
trend of open source software of having a base installation of a project and a lively ecosystem
of developers creating specialized tools that you can add or remove at will.

One of the first things you may want to do after installing
Julia is to add some useful packages. Recall that the first thing you need is to switch to the
package manager mode with ] .

You can check the packages that are currently installed with: **st**

You can add a package with: **add NameOfThePackage**

You can update a package with: **up NameOfThePackage**

You can remove a package with: **rm NameOfthePackage**

After a package has been installed, one use the statement:
** using NameOfThePackage **
to indicate that the code will use that package.

In the code cell below, the code installs the BenchmarkTools package,
and then uses it.

To benchmark a particular function, one use the\
** btime ** decorator, just before invoking a method.

In the code below, a matrix multiplication.

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   2752.7 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 13 seconds. 460 already precompiled.


  546.234 ms (2 allocations: 32.00 MiB)


# Types

Julia has variables, values, and types. A variable is a name bound to a value. Julia is case
sensitive: a is a different variable than A . In fact, as we will see below, the variable can
be nearly any combination of Unicode characters. A value is a content (1, 3.2, ”economics”,
etc.). Technically, Julia considers that all values are objects (an object is an entity with
some attributes). This makes Julia closer to pure object-oriented languages such as Ruby
than to languages such as C++, where some values such as floating points are not objects.
Finally, values have types (i.e., integer, float, boolean, string, etc.). A variable does not have
a type, its value has. Types specify the attributes of the content. Functions in Julia will
look at the type of the values passed as operands and decide, according to them, how we
can operate on the values (i.e., which of the methods available to the function to apply).
Adding 1+2 (two integers) will be different than summing 1.0+2.0 (two floats) because the
method for summing two integers is different from the method to sum two floats. In the base
implementation of Julia, there are 230 different methods for the function sum! You can list
them with the command methods() as in:

In [3]:
methods(+)

# 189 methods for generic function "+" from Base:
   [1] +(t::Dates.Time, dt::Dates.Date)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:22
   [2] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:85
   [3] +(x::Bool, z::Complex{Bool})
     @ complex.jl:305
   [4] +(x::Bool, y::Bool)
     @ bool.jl:166
   [5] +(x::Bool)
     @ bool.jl:163
   [6] +(x::Bool, z::Complex)
     @ complex.jl:312
   [7] +(x::Real, z::Complex{Bool})
     @ complex.jl:319
   [8] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
   [9] +(z::Complex{Bool}, x::Bool)
     @ complex.jl:306
  [10] +(z::Complex{Bool}, x::Real)
     @ complex.jl:320
  [11] +(z::Complex, x::Bool)
     @ complex.jl:313
  [12] +(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:501
  [13] +(a::BigInt, b::BigInt, c::BigInt)
     @ Base.GMP gmp.jl:541
  [14] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt)
     @ Base.GMP gmp.jl:542
  [15] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt)
     @ Base.GMP gmp.jl:543
  [16] +(x::BigInt, y::BigInt, rest::BigInt...)
     @ Base.GMP gmp.jl:683
  [17] +(c::BigInt, x::BigFloat)
     @ Base.MPFR mpfr.jl:468
  [18] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.GMP gmp.jl:549
  [19] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8})
     @ Base.GMP gmp.jl:555
  [20] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:334
  [21] +(x::Dates.CompoundPeriod, y::Dates.TimeType)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:362
  [22] +(x::Dates.CompoundPeriod, y::Dates.Period)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:332
  [23] +(B::BitMatrix, J::LinearAlgebra.UniformScaling)
     @ LinearAlgebra /usr/local/share/julia/stdlib/v1.10/LinearAlgebra/src/uniformscaling.jl:151
  [24] +(x::Dates.Date, y::Dates.Day)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:81
  [25] +(x::Dates.Date, y::Dates.Week)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:79
  [26] +(x::Dates.Date, y::Dates.Quarter)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:75
  [27] +(dt::Dates.Date, z::Dates.Month)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:56
  [28] +(dt::Dates.Date, y::Dates.Year)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:29
  [29] +(dt::Dates.Date, t::Dates.Time)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:21
  [30] +(::Missing, ::Missing)
     @ missing.jl:122
  [31] +(::Missing)
     @ missing.jl:101
  [32] +(x::Missing, y::Dates.AbstractTime)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:91
  [33] +(::Missing, ::Number)
     @ missing.jl:123
  [34] +(x::Dates.DateTime, y::Dates.Quarter)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:77
  [35] +(dt::Dates.DateTime, z::Dates.Month)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:49
  [36] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:25
  [37] +(x::Dates.DateTime, y::Dates.Period)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:83
  [38] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat)
     @ Base.MPFR mpfr.jl:619
  [39] +(x::BigFloat, y::BigFloat)
     @ Base.MPFR mpfr.jl:432
  [40] +(a::BigFloat, b::BigFloat, c::BigFloat)
     @ Base.MPFR mpfr.jl:606
  [41] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat)
     @ Base.MPFR mpfr.jl:612
  [42] +(x::BigFloat, c::BigInt)
     @ Base.MPFR mpfr.jl:463
  [43] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.MPFR mpfr.jl:439
  [44] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8})
     @ Base.MPFR mpfr.jl:447
  [45] +(x::BigFloat, c::Union{Float16, Float32, Float64})
 

# Polymorphic multiple dispatch

This application of different methods to a common function is known as polymorphic multiple
dispatch and it is one of the key concepts in Julia you need to understand.

Multiple dispatch is different from the overloading of operators existing in languages such as C++ because
it is determined at run time, not compilation time. Later, when we introduce composite types, we will see a
second difference: in Julia, methods are not defined within classes as you would do in most object-oriented
languages.

The previous paragraph may help to see why Julia is a strongly dynamically typed
programming language. Being a typed language means that the type of each value must be
known by the compiler at run time to decide which method to apply to that value. Being a
dynamically typed language means that such knowledge can be either explicit (i.e., declared
by the user) or implicit (i.e., deduced by Julia with an intelligent type inference engine from
the context it is used). Dynamic typing makes developing code with Julia flexible and fast:
you do not need to worry about explicitly type every value as you go along (i.e., declaring to
which type the value belongs). Being a strongly typed language means that you cannot use
a value of one type as another value, although you can convert it or let the compiler do it for
you. For example, Julia follows a promotion system where values of different types being
operated jointly are “promoted” to a common system: in the sum between an integer and a
float, the integer is “promoted” to float.10 You can, nevertheless, impose that the compiler
will not vary the type of a value to avoid subtle bugs in issues where the type is of critical
importance such as array indexing and, sometimes, to improve performance by providing
guidance to the JIT compiler on which methods to implement.

You do not need, though, to remember the type tree hierarchy, since Julia provides you
with commands to check the supertype (i.e., the type above a current type in the tree) and
subtype (i.e., the types below) of any given type:

In [2]:
supertype(Float64)

AbstractFloat

In [1]:

subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

You can always check the type of a variable with

In [4]:
a = 10
typeof(a)

Int64

## Variables

By default, integers values will be Int64 and floating point values will be
Float64 , but we also have shorter and longer types. Particularly
useful for computations with absolute large numbers (this happens sometimes, for example,
when evaluating likelihood functions), we have BigFloat. In the unlikely case that BigFloat
does not provide you with enough precission, Julia can use the GNU Multiple Precision
arithmetic (GMP) (https://gmplib.org/) and the GNU MPFR Libraries (http://www.
mpfr.org/).

In [8]:
a = 3 # integer
a = 0x3 # unsigned integer, hexadecimal base
a = 0b11 # unsigned integer, binary base
a = 3.0 # Float64
a = 4 + 3im # imaginary
a = complex(4,3) # same as above
a = true # boolean
a = "String" # string

"String"

# Minimum and Maximum for every type
You can check the minimum and maximum value every type can store with the functions
typemin() and typemax() , the machine precision of a type with eps() and, if it is a floating point, the effective bits in its mantissa by precision() . For example, for a
Float64 :

In [9]:
typemin(Float64) # returns -Inf (just a convention)
typemin(Float64) # returns Inf (just a convention)
eps(Float64) # returns 2.22e-16
precision(Float64) # returns 53

53

Larger or smaller numbers than the limits will return an overflow error. You can also check
the binary representation of a value:

In [10]:
a = 1
bitstring(a) # binary representation of a

"0000000000000000000000000000000000000000000000000000000000000001"

In [ ]:
import Pkg; Pkg.add("CUDA")
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   1976.1 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 3 seconds. 460 already precompiled.


Fri Apr  4 16:37:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />